In [ ]:
from training_rl.offline_rl.custom_envs.custom_envs_registration import register_grid_envs
import warnings
from training_rl.offline_rl.load_env_variables import load_env_variables
from training_rl.offline_rl.scripts.visualizations.utils import snapshot_env
from training_rl.offline_rl.custom_envs.utils import Grid2DInitialConfig, InitialConfigCustom2DGridEnvWrapper
from training_rl.offline_rl.custom_envs.custom_2d_grid_env.obstacles_2D_grid_register import ObstacleTypes
from training_rl.offline_rl.custom_envs.custom_envs_registration import CustomEnv, RenderMode
import gymnasium as gym
from training_rl.offline_rl.behavior_policies.behavior_policy_registry import BehaviorPolicyType
from training_rl.offline_rl.generate_custom_minari_datasets.generate_minari_dataset_grid_envs import \
    create_combined_minari_dataset
from training_rl.offline_rl.offline_policies.offpolicy_rendering import offpolicy_rendering
from training_rl.offline_rl.offline_trainings.policy_config_data_class import TrainedPolicyConfig
from training_rl.offline_rl.offline_policies.policy_registry import PolicyName
from training_rl.offline_rl.offline_trainings.offline_training import offline_training
import torch
from training_rl.offline_rl.offline_trainings.policy_config_data_class import get_trained_policy_path
import os
from training_rl.offline_rl.offline_trainings.restore_policy_model import restore_trained_offline_policy
from training_rl.offline_rl.utils import load_buffer_minari

load_env_variables()

warnings.filterwarnings("ignore")
register_grid_envs()

### Exercise

**In this exercise, we'll evaluate the distributional shift in the CQL and BCQ algorithms and how they deal with it.**

As mentioned earlier, regularization methods such as CQL are a suitable choice when prioritizing safety in your agent's behavior. However, if your focus is primarily on achieving an optimal solution with fewer constraints on safety, methods like BCQ may be more suitable.

In this exercise we will start from (0,0) and we will try to reach the target at (4,7) but the target is protected by a wall. We will collect data again from suboptimal policies as shown below (section 1.2).

#### Environment

In [ ]:
ENV_NAME = CustomEnv.Grid_2D_8x8_discrete

# Env. Config.
OBSTACLE = ObstacleTypes.door_object_8x8
INITIAL_STATE = (0, 0)
FINAL_STATE = (4, 7)

env_2D_grid_initial_config = Grid2DInitialConfig(
    obstacles=OBSTACLE,
    initial_state=INITIAL_STATE,
    target_state=FINAL_STATE,
)

env = InitialConfigCustom2DGridEnvWrapper(gym.make(ENV_NAME, render_mode=RenderMode.RGB_ARRAY_LIST), 
                                          env_config=env_2D_grid_initial_config)
snapshot_env(env)

#### Configure the two datasets

In [ ]:
IDENTIFIER_COMBINED_DATASETS = "_conservative_test"

# Dataset I
#BEHAVIOR_POLICY_I = BehaviorPolicyType.behavior_8x8_moves_downwards_within_strip
BEHAVIOR_POLICY_I = BehaviorPolicyType.behavior_8x8_grid_suboptimal_0_0_to_4_7
DATA_SET_IDENTIFIER_I = "_suboptimal"
NUM_STEPS_I = 500

# Dataset II
#BEHAVIOR_POLICY_II = BehaviorPolicyType.behavior_8x8_eps_greedy_4_0_to_7_7
BEHAVIOR_POLICY_II = BehaviorPolicyType.random#behavior_8x8_grid_deterministic_0_0_to_4_7
DATA_SET_IDENTIFIER_II = "_random"
NUM_STEPS_II = 8000

#### Create Minari combined dataset

In [ ]:
config_combined_data = create_combined_minari_dataset(
        env_name=ENV_NAME,
        dataset_identifiers = (DATA_SET_IDENTIFIER_I, DATA_SET_IDENTIFIER_II),
        num_collected_points = (NUM_STEPS_I, NUM_STEPS_II),
        behavior_policy_names = (BEHAVIOR_POLICY_I, BEHAVIOR_POLICY_II),
        combined_dataset_identifier = "combined_dataset",
        env_2d_grid_initial_config = env_2D_grid_initial_config,
)
buffer_data = load_buffer_minari(config_combined_data.data_set_name)
data_size = len(buffer_data)

#### Rendering behavioral policy

In [ ]:
# Policy I
offpolicy_rendering(
    env_or_env_name=ENV_NAME,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    behavior_policy_name=BEHAVIOR_POLICY_I,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=1000,
)

In [ ]:
# Policy II
offpolicy_rendering(
    env_or_env_name=ENV_NAME,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    behavior_policy_name=BEHAVIOR_POLICY_II,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=1000,
)

#### Choose your policy

In [ ]:
# The model policy to be trained.
POLICY_NAME = PolicyName.bcq_discrete


NAME_EXPERT_DATA = config_combined_data.data_set_name
# TrainedPolicyConfig is a handy object that will help us to deal with the policy configuration data.
offline_policy_config = TrainedPolicyConfig(
    name_expert_data=NAME_EXPERT_DATA,
    policy_name=POLICY_NAME,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    device="cpu",
)

#### Training

In [ ]:
NUM_EPOCHS = 20
BATCH_SIZE = 128
STEP_PER_EPOCH = 0.1*data_size

# After every epoch we will collect some test statistics from the policy from NUMBER_TEST_ENVS independent envs.
NUMBER_TEST_ENVS = 1
EXPLORATION_NOISE = True
SEED = None #1626


# Run the training
offline_training(
    offline_policy_config=offline_policy_config,
    num_epochs = NUM_EPOCHS,
    number_test_envs=NUMBER_TEST_ENVS,
    step_per_epoch=STEP_PER_EPOCH,
    restore_training=False,
)

#### Restore policy

In [ ]:
POLICY_FILE = "policy_best_reward.pth"
# restore a policy with the same configuration as the one we trained.
policy = restore_trained_offline_policy(offline_policy_config)
# load the weights
name_expert_data = offline_policy_config.name_expert_data
log_name = os.path.join(name_expert_data, POLICY_NAME)
log_path = get_trained_policy_path(log_name)
policy.load_state_dict(torch.load(os.path.join(log_path, POLICY_FILE), map_location="cpu"))


#### Render trained policy

In [ ]:
offpolicy_rendering(
    env_or_env_name=env,
    render_mode=RenderMode.RGB_ARRAY_LIST,
    policy_model=policy,
    env_2d_grid_initial_config=env_2D_grid_initial_config,
    num_frames=1000,
    imitation_policy_sampling=False
)

### Exercise 2:

a) Remove the obstacle. What do you think are going to be the results?

b) Modify the parameters related to distributional shift in BCQ and CQL, and observe their impact on out-of-distribution behavior.